In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import datetime
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.utils import shuffle


def readDataSets():
    dataSetFridge = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Fridge.csv')
    dataSetGarageDoor = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Garage_Door.csv')
    dataSetGPS = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_GPS_Tracker.csv')
    dataSetModbus = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Modbus.csv')
    dataSetMotionLight = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Motion_Light.csv')
    dataSetThermostat = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Thermostat.csv')
    dataSetWeahter = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Weather.csv')

    dataSetFridge['temp_condition'] = dataSetFridge['temp_condition'].str.strip()
    dataSetGarageDoor['door_state'] = dataSetGarageDoor['door_state'].str.strip()
    dataSetMotionLight['light_status'] = dataSetMotionLight['light_status'].str.strip()
    
    dataSetFridge = cutSize(dataSetFridge)
    dataSetGarageDoor = cutSize(dataSetGarageDoor)
    dataSetGPS = cutSize(dataSetGPS)
    dataSetModbus = cutSize(dataSetModbus)
    dataSetMotionLight = cutSize(dataSetMotionLight)
    dataSetThermostat = cutSize(dataSetThermostat)
    dataSetWeahter = cutSize(dataSetWeahter)
    
    
    dataSetRawLoad = pd.concat([dataSetFridge, dataSetGarageDoor, dataSetGPS, dataSetModbus, dataSetMotionLight, dataSetThermostat, dataSetWeahter])
    #dataSetRawLoad = pd.concat([dataSetFridge, dataSetGarageDoor, dataSetGPS, dataSetModbus])
    
    print('dataSetRawLoad: ', dataSetRawLoad.shape)
    return dataSetRawLoad

In [ ]:
# Load the data
categorical_features = ['type', 'door_state','sphone_signal', 'light_status','temp_condition']
quantitative_features = ['FC1_Read_Input_Register','FC2_Read_Discrete_Value','FC3_Read_Holding_Register','FC4_Read_Coil','current_temperature',
                       'fridge_temperature','humidity','latitude','FC4_Read_Coil','longitude',
                      'motion_status','pressure','temperature','thermostat_status']
#categorical_features = ['type', 'door_state','sphone_signal', 'temp_condition']
#quantitative_features = ['FC1_Read_Input_Register','FC2_Read_Discrete_Value','FC3_Read_Holding_Register','FC4_Read_Coil',
#                        'fridge_temperature',
#                         #'humidity' ,
#                         'latitude','FC4_Read_Coil','longitude',
#                        #'pressure',
#                        # 'temperature'
#                        ]
features = categorical_features + quantitative_features

def datapreprocessingShuffle(data):
               
    # Feature scaling
    for i in quantitative_features :
            #scaler = StandardScaler()
            #data[i] = scaler.fit_transform(data[[i]])
            data[i] = data[i]
            
    # Encoding categorical features    
    for i in categorical_features : 
        labelencoder=LabelEncoder()
        data[i]=labelencoder.fit_transform(data[i])   
    
    data = shuffle(data).reset_index(drop=True) 
    
    Y = data.loc[:,'type']
    X = data.drop(['type'],axis=1) 
    
    return(X,Y)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

def funcTransform(d):
    d1 = d.iloc[:,:-1]
    
    dN =pd.DataFrame()
    label=LabelEncoder()
    for c in  d1.columns:
        if(d[c].dtype=='object'):
            dN[c]=label.fit_transform(d1[c])
        else:
            dN[c]=d1[c]
            
    c = dN.columns
    ff_arr = preprocessing.normalize(dN)
    df1 = pd.DataFrame(data=ff_arr, columns=c)
    df1['label'] = d['type']
    return df1

def funcTransform1(d):
    dN =pd.DataFrame()
    label=LabelEncoder()
    for c in  d.columns:
        if(d[c].dtype=='object'):
            dN[c]=label.fit_transform(d[c])
        else:
            dN[c]=d[c]
            
    return dN

def cutSize(df):
    newdf1 = df[df['type'] == 'password'].iloc[0:30000,:]
    newdf2 = df[df['type'] == 'injection'].iloc[0:30000,:]
    #newdf3 = df[df['type'] == 'backdoor'].iloc[0:5000,:]
    #newdf4 = df[df['type'] == 'ddos'].iloc[0:15000,:]
    #newdf5 = df[df['type'] == 'ransomware'].iloc[0:15000,:]
    #newdf6 = df[df['type'] == 'xss'].iloc[0:15000,:]
    #newdf7 = df[df['type'] == 'scanning'].iloc[0:15000,:]
    n = pd.concat([newdf1, newdf2]).sample(frac=1).reset_index(drop=True)
    return n

In [20]:
df = readDataSets()
df1 = df.copy()
df2 = df.copy()
dfm = pd.concat([df, df1, df2]).sample(frac=1).reset_index(drop=True)
print(dfm.shape)
dfo = cutSize(dfm)
# Pre-processing datset
datacopy = dfo.copy()
X, y = datapreprocessingShuffle(datacopy) 

X = X.fillna(X.median())

In [38]:
pd.DataFrame(dfo['type'].value_counts())[:]

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score,plot_confusion_matrix
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from statistics import mean 
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.ensemble import VotingClassifier
from matplotlib import pyplot
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
import time
from sklearn.model_selection import RepeatedStratifiedKFold



# This method will give the max value item in list
def my_max_by_weight(sequence):
    if not sequence:
        raise ValueError('empty sequence')
    maximum = sequence[0]
    for item in sequence:
        if item[0] > maximum[0]:
            maximum = item
    return maximum



# This will give the classification results using Logistic Regression model
def LogisticRegressionAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    cS = np.linspace(1, 100, 100, endpoint=True, dtype= int)
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for c in cS:
        dt = LogisticRegression(C=c, random_state=0)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, c])
    # using C that has most accuracy in the range in the list
    dt = LogisticRegression(C=my_max_by_weight(cValueList)[1], random_state=0)
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj


# This will give the classification results using KNeighbors  model
def KNeighborsAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    n_neighbors = np.linspace(1, 100, 50, endpoint=True, dtype= int)
    # object of train and testing arrays from TFIDF vectorizer
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with n_neighbors value for hyper parameter
    for n_neighbor in n_neighbors:
        dt = KNeighborsClassifier(n_neighbors=n_neighbor,metric='minkowski',p=2)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, n_neighbor])
    # using n_neighbors that has most accuracy in the range in the list
    dt = KNeighborsClassifier(n_neighbors=my_max_by_weight(cValueList)[1],metric='minkowski',p=2)
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj

# This will give the classification results using Logistic Regression model
def DTAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    max_depths = np.linspace(1, 100, 100, endpoint=True, dtype= int)
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for max_depth in max_depths:
        dt = DecisionTreeClassifier(max_depth=max_depth, random_state=0)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, max_depth])
    # using C that has most accuracy in the range in the list
    dt = DecisionTreeClassifier(max_depth=my_max_by_weight(cValueList)[1], random_state=0)
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj


# This will give the classification results using Logistic Regression model
def RFAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    max_depths = np.linspace(1, 100, 100, endpoint=True, dtype= int)
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for max_depth in max_depths:
        dt = RandomForestClassifier(max_depth=max_depth, random_state=0)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, max_depth])
    # using C that has most accuracy in the range in the list
    dt = RandomForestClassifier(max_depth=my_max_by_weight(cValueList)[1], random_state=0)
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj

def GBAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    max_depths = np.linspace(100, 120, 10, endpoint=True, dtype= int)
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for max_depth in max_depths:
        dt = GradientBoostingClassifier(n_estimators=max_depth)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, max_depth])
    # using C that has most accuracy in the range in the list
    dt = GradientBoostingClassifier(n_estimators=my_max_by_weight(cValueList)[1])
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj

def NBAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    max_depths = np.logspace(0,-9, num=10)
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for max_depth in max_depths:
        dt = GaussianNB(var_smoothing=max_depth)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, max_depth])
    # using C that has most accuracy in the range in the list
    dt = GaussianNB(var_smoothing=my_max_by_weight(cValueList)[1])
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj





# This will give the classification results using Logistic Regression model
def AdaBoostClassifierAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    #max_depths = np.linspace(1, 10, 10, endpoint=True, dtype= int)
    n_trees = [10, 50, 100, 500, 1000, 5000]
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for n in n_trees:
        dt = AdaBoostClassifier(n_estimators=n,learning_rate=1)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, n])
    # using C that has most accuracy in the range in the list
    dt = AdaBoostClassifier(n_estimators=my_max_by_weight(cValueList)[1])
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj

# This will give the classification results using Logistic Regression model
def XGBClassifierAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    max_depths =np.linspace(1, 100, 100, endpoint=True, dtype= int)
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for max_depth in max_depths:
        dt = XGBClassifier(max_depth=max_depth, max_delta_step=0)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, max_depth])
    # using C that has most accuracy in the range in the list
    dt = XGBClassifier(max_depth=my_max_by_weight(cValueList)[1], max_delta_step=0)
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj

# This will give the classification results using Logistic Regression model
def SVCAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    cS = np.linspace(1, 100, 100, endpoint=True, dtype= int)
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for c in cS:
        dt = SVC(C = c , kernel="rbf",random_state=0)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, c])
    # using C that has most accuracy in the range in the list
    dt = SVC(C=my_max_by_weight(cValueList)[1], kernel="rbf",random_state=0)
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj

# This will give the classification results using Logistic Regression model
def LDAAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    cS = ['svd', 'lsqr', 'eigen']
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for c in cS:
        dt = LinearDiscriminantAnalysis(solver = c)
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, c])
    # using C that has most accuracy in the range in the list
    dt = LinearDiscriminantAnalysis(solver=my_max_by_weight(cValueList)[1])
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    score = f1_score(y_test, y_pred, average='weighted')
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy}
    return resultObj

def SoftVotingAccuracy(X_train, X_test, y_train, y_test):
    # create a variable for the range from 1 to 100
    models = list()
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('CART', DecisionTreeRegressor()))
    models.append(('NB', GaussianNB()))
    models.append(('MLP', MLPClassifier()))
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('LR', LogisticRegression()))
    models.append(('SVM', svm.SVC()))
    models.append(('AdaBoost', AdaBoostClassifier()))
    models.append(('GradientBoosting', GradientBoostingClassifier()))
    models.append(('XGB', XGBClassifier()))
    # properties initialization
    train_results = []
    test_results = []
    cValueList = []
    # loop that applied range on the model one by one and return a list of accuracies with C value for hyper parameter
    for c in models:
        dt = VotingClassifier(estimators=c, voting='soft')
        dt.fit(X_train,y_train)
        pred_result = dt.predict(X_test)
        score2 = dt.score(X_test, y_test)
        cValueList.append([score2, c])
    # using C that has most accuracy in the range in the list
    dt = VotingClassifier(estimators=my_max_by_weight(cValueList)[1], voting='soft')
    # fit the train and test
    dt.fit(X_train, y_train)
    # plot confusion matrix 
    plot_confusion_matrix(dt, X_test, y_test)
    plt.show()
    y_pred = dt.predict(X_test)
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    score = f1_score(y_test, y_pred)
    resultObj = {"precision":precision,"recall":recall,"f1score":score,"accuracy":accuracy,"cm":cm}
    return resultObj

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection  import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

# We use a utility to generate artificial classification data.
X = X.values
y = y.values
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    lr = LogisticRegressionAccuracy(X_train, X_test, y_train, y_test)
    print("LR", lr)
    knn = KNeighborsAccuracy(X_train, X_test, y_train, y_test)
    print("KNN", knn)
    dt = DTAccuracy(X_train, X_test, y_train, y_test)
    print("DT", dt)
    rf = RFAccuracy(X_train, X_test, y_train, y_test)
    print("RF", rf)
    gb = GBAccuracy(X_train, X_test, y_train, y_test)
    print("GB", gb)
    nb = NBAccuracy(X_train, X_test, y_train, y_test)
    print("NB", nb)
    ada = AdaBoostClassifierAccuracy(X_train, X_test, y_train, y_test)
    print("ADA", ada)
    xgb = XGBClassifierAccuracy(X_train, X_test, y_train, y_test)
    print("XGB", xgb)
    svm = SVCAccuracy(X_train, X_test, y_train, y_test)
    print("SVM", svm)
    lda = LDAAccuracy(X_train, X_test, y_train, y_test)
    print("LDA", lda)
    sv = SoftVotingAccuracy(X_train, X_test, y_train, y_test)
    print("gb", sv)

In [ ]:
# exclude scanning
from sklearn.datasets import make_classification
from sklearn.model_selection  import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

# We use a utility to generate artificial classification data.
X = X.values
y = y.values
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.25, random_state=0)
sss.get_n_splits(X, y)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    lr = LogisticRegressionAccuracy(X_train, X_test, y_train, y_test)
    print("LR", lr)
    knn = KNeighborsAccuracy(X_train, X_test, y_train, y_test)
    print("KNN", knn)
    dt = DTAccuracy(X_train, X_test, y_train, y_test)
    print("DT", dt)
    rf = RFAccuracy(X_train, X_test, y_train, y_test)
    print("RF", rf)
    gb = GBAccuracy(X_train, X_test, y_train, y_test)
    print("GB", gb)
    nb = NBAccuracy(X_train, X_test, y_train, y_test)
    print("NB", nb)
    ada = AdaBoostClassifierAccuracy(X_train, X_test, y_train, y_test)
    print("ADA", ada)
    xgb = XGBClassifierAccuracy(X_train, X_test, y_train, y_test)
    print("XGB", xgb)
    svm = SVCAccuracy(X_train, X_test, y_train, y_test)
    print("SVM", svm)
    lda = LDAAccuracy(X_train, X_test, y_train, y_test)
    print("LDA", lda)
      sv = SoftVotingAccuracy(X_train, X_test, y_train, y_test)
    print("gb", sv)

In [ ]:
# exclude scanning
from sklearn.datasets import make_classification
from sklearn.model_selection  import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

# We use a utility to generate artificial classification data.
X = X.values
y = y.values
sss = StratifiedShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    lr = LogisticRegressionAccuracy(X_train, X_test, y_train, y_test)
    print("LR", lr)
    knn = KNeighborsAccuracy(X_train, X_test, y_train, y_test)
    print("KNN", knn)
    dt = DTAccuracy(X_train, X_test, y_train, y_test)
    print("DT", dt)
    rf = RFAccuracy(X_train, X_test, y_train, y_test)
    print("RF", rf)
    gb = GBAccuracy(X_train, X_test, y_train, y_test)
    print("GB", gb)
    nb = NBAccuracy(X_train, X_test, y_train, y_test)
    print("NB", nb)
    ada = AdaBoostClassifierAccuracy(X_train, X_test, y_train, y_test)
    print("ADA", ada)
    xgb = XGBClassifierAccuracy(X_train, X_test, y_train, y_test)
    print("XGB", xgb)
    svm = SVCAccuracy(X_train, X_test, y_train, y_test)
    print("SVM", svm)
    lda = LDAAccuracy(X_train, X_test, y_train, y_test)
    print("LDA", lda)
      sv = SoftVotingAccuracy(X_train, X_test, y_train, y_test)
    print("gb", sv)